In [ ]:
#introduccion a generacion de texto con red neuronal
#las redes neuronales recurrentes son muy utilidas cuando se va a la procesion dee datos secuenciales como texto.
#En esta practica vamos a usar una red neuronal LSTM(Long short term memory o termino de memoria de corta a larga)
#para enseñar a nuestra computadora a escribir textos como shakespeare.

In [2]:
#Primero vamos a cargar las liberarias que vamos a emplear.
#En este caso Tensorflow, Numpy y Random.
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [3]:
#Primero que nada necesitamos una cantidad larga de enunciados de texto de shakespeare para entrenar el modelo.
#por lo que vamos podemos yasea descargar uno o hacer uso de uno de una pagina.
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')


In [4]:
#Ahora bien lo siguiente es convertir el formato del texto en algo que la red neuronal pueda entender.
#Por lo que necesitamos convertir todos esos valores a datos numericos.
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
#en caso de que no se quiera usar todo el texto o una parte limitada, se puede emplear un .lower(),
#para reducir todo lo que se vaya a convertir, esto con el fin de no demorar en convertir todo el fragmento de texto.
#A parte de producir un mejor resultado.

In [5]:
#Aqui vamos a seleccionar a todos los caracteres de numero de 300,000 caracteres hasta 800,000.
#Estamos produciendo un total de 500,000 caracteres, los cuales deberian ser suficientes para resultados muy decentes.
text = text[300000:800000]

In [6]:
#Ahora vamos a crear un tipo de conjunto de todos los caracteres unicos que ocurren en el texto.
#En un no conjunto aparecen mas de una vez,lo cual es una forma de filtrar los caracteres.
characters = sorted(set(text))
#Despues de eso, definimos dos estructuras para convertir los valores.
#Ambas son diccionarios que enumeran los caracteres.
char_to_index = dict((c, i) for i, c in enumerate(characters))
#En el primero, los caracteres son las llaves y los indices son los valores.
index_to_char = dict((i, c) for i, c in enumerate(characters))
#En el segundo es viceversa.

In [7]:
#En el siguiente paso, vamos a definir que tan largo debe ser la secuencia y tambien cuantos caracteres
#seguira para empezar el siguiente enunciado.
#Lo que trataremos de hacer aqui es tomar los enunciados y entonces guardarlos en la siguiente letra como un dato de entrenamiento.
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_charl = []

#iteramos a traves de todo el texto y reunir todos los enunciados y sus siguientes caracteres.
#Este es el dato de entrenamiento para nuestra red reuronal.
#Ahora necesitamos convertirlo en un formato numerico.
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_charl.append(text[i + SEQ_LENGTH])

In [8]:
#Se crearan dos arreglos de Numpy todos de zeros.
#El tipo de datos es booleano. Donde un caracter aparezca en varios enunciados en una posicion certera.
#lo coloraremos a uno o a un True. Tendremos una dimension para los enunciados.
#Una dimension para la posicion de los caracteres dentro de los enunciados y una dimension para especificar
#Que caracter esta en esa posicion.

x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_charl[i]]] = 1

In [9]:
#Creacion de modelo
#Usaremos Sequential para nuestro modelo,activacion, densidad y LSTM para nuestros layers
#y RMSprop para la optimizacion durante la compilacion de nuestro modelo.
#LSTM soporta para long-short-term-memory y este tipo de estracto recurrente n las redes neuronales.
#podria ser llamado la memoria de nuestro modelo.Este es crucial,desde que estamos tratando con datos secuenciales.
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [11]:
model.fit(x,y, batch_size=256, epochs=4)
#Ahora compilaremos el modelo y entrenarlo con nuestros datos de entrenamiento que preparamos por encima.
#Escogemos el tamaño de lote de 256(el cual puedes cambiar si quieres) y 4 epoca. Esto significa que nuestro modelo vera el mismo tipo de dato 4 veces.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 166654 samples
Epoch 1/4
166654/166654 [==============================] - 72s 430us/sample - loss: 2.1257
Epoch 2/4
166654/166654 [==============================] - 79s 474us/sample - loss: 1.7156
Epoch 3/4
166654/166654 [==============================] - 80s 479us/sample - loss: 1.5887
Epoch 4/4
166654/166654 [==============================] - 85s 513us/sample - loss: 1.5227


In [12]:
#se guarda el modelo creado 
model.save('textgenerator.model') 

In [13]:
#se carga el modelo
model = tf.keras.models.load_model('textgenerator.model')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
#nuestro modelo esta ahora entrenado pero solo las salidas de las posibilidades para el siguiente caracter.
#Entonces necesitamos alguna funcion adicional para hacer que nuestro guion genere un texto razonable.
#Esta funcion de ayuda llamada nuestra puedes usarla de la pagina de keras.
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
#Es basicamente elegir uno de los caracteres de la salida. como parametros tomara el resultado de la prediccion y una temperatura.
#Esta temperatura indica que tan riesgoso el elegir debe ser.
#Si tenemos una temperatura alta, elegiremos uno de los caracteres menos probables.
#Una temperatura baja causara una eleccion conservativa.

In [19]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [21]:
print('--------0.2---------')
print(generate_text(300,0.2))
print('--------0.4---------')
print(generate_text(300,0.4))
print('--------0.6---------')
print(generate_text(300,0.6))
print('--------0.8---------')
print(generate_text(300,0.8))
print('--------1---------')
print(generate_text(300,1.0))

--------0.2---------
d the hand of war,
this happy breed of my lord, the banish
and man and my love, that she would not for her soul.

king richard ii:
then the did my soul in the words him stand
and then the words him not the words the words,
and thou wilt the words of the words of his comes
and be with the words a bust to the words,
when the words the words
--------0.4---------
to let you understand,
if case some one to the words have strees,
and then where we well to his makes for him now,
stay the distand richard his sir, as which we warwick
as soul my words me with a roush the warwick,
and therefore and will do the cate the words,
where is not the day with my matter of his fair
the will my lord, there is a be
--------0.6---------
royalties and to beg
enfranchisement immor livest the sun the eneminis
make for her will strong with surper'd of wence,
but what is become your fortunes good,
and for the sweat the receity, for this been,
as we are with meet when thou ween he that i will 